# Clustering Professionals

In [1]:
# import statements for all of the packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

C:\Users\aishwarya\Anaconda3\lib\site-packages\ipykernel\parentpoller.py:116: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  ipython-dev@scipy.org""")


In [2]:
# Loading data and printing a few lines 
df = pd.read_csv('dataset/professionals.csv')
df.head()

,professionals_id,professionals_location,professionals_industry,professionals_headline,professionals_date_joined
0,9ced4ce7519049c0944147afb75a8ce3,NaN,NaN,NaN,2011-10-05 20:35:19 UTC+0000
1,f718dcf6d2ec4cb0a52a9db59d7f9e67,NaN,NaN,NaN,2011-10-05 20:49:21 UTC+0000
2,0c673e046d824ec0ad0ebe012a0673e4,"New York, New York",NaN,NaN,2011-10-18 17:31:26 UTC+0000
3,977428d851b24183b223be0eb8619a8c,"Boston, Massachusetts",NaN,NaN,2011-11-09 20:39:29 UTC+0000
4,e2d57e5041a44f489288397c9904c2b2,NaN,NaN,NaN,2011-12-10 22:14:44 UTC+0000


In [3]:
df.shape

(28152, 5)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28152 entries, 0 to 28151
Data columns (total 5 columns):
professionals_id             28152 non-null object
professionals_location       25054 non-null object
professionals_industry       25576 non-null object
professionals_headline       26085 non-null object
professionals_date_joined    28152 non-null object
dtypes: object(5)
memory usage: 1.1+ MB


In [5]:
df['professionals_location'].unique().size

2583

In [6]:
df['professionals_date_joined'] = pd.to_datetime(df['professionals_date_joined'])

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28152 entries, 0 to 28151
Data columns (total 5 columns):
professionals_id             28152 non-null object
professionals_location       25054 non-null object
professionals_industry       25576 non-null object
professionals_headline       26085 non-null object
professionals_date_joined    28152 non-null datetime64[ns]
dtypes: datetime64[ns](1), object(4)
memory usage: 1.1+ MB


In [8]:
df['professionals_state'] = df['professionals_location'].str.rsplit(",", 1).str[-1].str.strip()
# df.drop(columns=['professionals_location'], inplace=True)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28152 entries, 0 to 28151
Data columns (total 6 columns):
professionals_id             28152 non-null object
professionals_location       25054 non-null object
professionals_industry       25576 non-null object
professionals_headline       26085 non-null object
professionals_date_joined    28152 non-null datetime64[ns]
professionals_state          25054 non-null object
dtypes: datetime64[ns](1), object(5)
memory usage: 1.3+ MB


In [10]:
unique_states = pd.Series(df['professionals_state'].unique().astype(str))
unique_states

0                                   nan
1                              New York
2                         Massachusetts
3                             Wisconsin
4                               Vermont
5                            Washington
6                            California
7                              Michigan
8                             Tennessee
9                                 Texas
10                                India
11                       United Kingdom
12                         Pennsylvania
13                              Indiana
14                              Arizona
15                             Virginia
16                               Canada
17                        United States
18                              Florida
19                             Illinois
20                               Beburg
21                            Minnesota
22                           Cape Verde
23                              Georgia
24                                 Oman


In [11]:
unique_states[unique_states.str.contains('New York')==True]

1                                         New York
170    Greater Chicago Area; Greater New York Area
190                     Greater New York City Area
215                                  New York Area
dtype: object

In [12]:
# counts = df['professionals_state'].value_counts()
states = unique_states.copy()

def merge_duplicates(state):
    count = 0
    for s in states:
        if state in s:
            count = count + 1
        if count == 2:
            states.drop(states[states == s].index, inplace=True)
            count = 1

In [13]:
unique_states.apply(merge_duplicates)
states.size

239

In [14]:
temp_pref_states = df['professionals_state'].astype(str).copy()

In [15]:
def update_states(state):
    global temp_pref_states
    def check_state(s):
        if state in s:
            return state
        else:
            return s
    temp_pref_states = temp_pref_states.apply(check_state)

In [16]:
for state in temp_pref_states:
    update_states(state)
# temp_pref_states = temp_pref_states.apply(update_states)
temp_pref_states

0                              na
1                              na
2                        New York
3                   Massachusetts
4                              na
5                              na
6                              na
7                              na
8                              na
9                              na
10                      Wisconsin
11                             na
12                             na
13                             na
14                             na
15                       New York
16                             na
17                             na
18                        Vermont
19                       New York
20                             na
21                             na
22                             na
23                             na
24                             na
25                             na
26                             na
27                     Washington
28                  Massachusetts
29            

In [17]:
temp_pref_states.value_counts()

na                             4414
Texas                          2984
California                     2921
New York                       2238
Washington                     1259
India                          1228
Massachusetts                   940
Florida                         875
Georgia                         808
Illinois                        711
Pennsylvania                    679
San Francisco Bay Area          513
New Jersey                      454
Maryland                        416
Michigan                        412
Greater Atlanta Area            383
Dallas/Fort Worth Area          368
Greater Los Angeles Area        364
Greater Chicago Area            298
Virginia                        296
Oregon                          288
Greater Boston Area             280
United States                   263
Ohio                            226
Greater Seattle Area            218
Greater Philadelphia Area       204
United Kingdom                  203
Connecticut                 

In [18]:
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(min_df = 0, max_df=0.5, stop_words = "english", ngram_range = (1,3))
# vec = vectorizer.fit(df['professionals_state'].astype(str))   # train vec using list1
# vectorized = vec.transform(df['professionals_state'].astype(str))   # transform list1 using vec
vec = vectorizer.fit(temp_pref_states.astype(str))   # train vec using list1
vectorized = vec.transform(temp_pref_states.astype(str))   # transform list1 using vec

km = KMeans(n_clusters=240, init='k-means++', n_init=10, max_iter=500, tol=0.0001, precompute_distances=True, verbose=0, random_state=None, n_jobs=1)

km.fit(vectorized)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=500,
    n_clusters=240, n_init=10, n_jobs=1, precompute_distances=True,
    random_state=None, tol=0.0001, verbose=0)

In [19]:
temp = pd.Series(km.labels_)
temp.value_counts()

1      4414
3      2984
2      2921
4      2238
6      1259
5      1228
7       940
10      875
12      808
9       711
13      679
70      513
17      454
15      416
0       412
14      383
18      368
8       364
23      298
20      296
19      288
22      280
21      263
24      226
31      218
25      204
28      203
16      194
29      177
43      159
       ... 
236       1
189       1
205       1
221       1
190       1
206       1
222       1
191       1
217       1
232       1
210       1
216       1
226       1
195       1
211       1
227       1
196       1
212       1
228       1
197       1
213       1
229       1
198       1
214       1
230       1
199       1
215       1
207       1
200       1
231       1
Length: 237, dtype: int64